<h1 style='text-align:center'> Iranian Telecom Churn </h1>
<h3 style='text-align:center'> ... </h3>

---
## Data Preparation

### Data Source

| Category | Description |
| --- | --- |
| URL | [University of California, Irvine (UCI) Machine Learning Repository: Iranian Churn](https://archive.ics.uci.edu/dataset/563/iranian+churn+dataset) |
| License | [Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/legalcode) |


### Data Context

| Category | Description |
| --- | --- |
| Dataset Characteristics | Multivariate |
| Subject Area | Business |
| Associated Tasks | Classification, Regression |
| Feature Type | Integer, Binary, Continuous |
| # of Instances | 3150 |
| # of Features | 13 |

This dataset is randomly collected from an Iranian telecom company's database over a period of 12 months. A total of 3150 rows of data, each representing a customer, bear information for 13 columns. The attributes that are in this dataset are call failures, frequency of SMS, number of complaints, number of distinct calls, subscription length, age group, the charge amount, type of service, seconds of use, status, frequency of use, and Customer Value.

All of the attributes except for attribute churn is the aggregated data of the first 9 months. The churn labels are the state of the customers at the end of 12 months. The three months is the designated planning gap.

### Data Dictionary

| <div style="width:110px"> Variable Type | Role | Type | Description |
| --- | --- | --- | --- |
| Call Failure | Feature | Integer | Number of call failures |
| Complains | Feature | Binary | Binary attribute (0: No complaint, 1: Complaint) |
| Subscription Length | Feature | Integer | Total months of subscription |
| Charge Amount	| Feature | Integer	| Ordinal attribute (0: lowest amount, 9: highest amount) |
| Seconds of Use | Feature	| Integer | Total seconds of calls |
| Frequency of use | Feature | Integer | Total number of calls |
| Frequency of SMS | Feature | Integer | Total number of text messages |
| Distinct Called Numbers | Feature | Integer | Total number of distinct phone calls |
| Age Group | Feature | Integer | Ordinal attribute (1: younger age, 5: older age) |
| Tariff Plan | Feature	| Binary | Binary attribute (1: Pay as you go, 2: contractual) |
| Status | Feature | Binary | Binary attribute (1: active, 2: non-active) |
| Age | Feature | Integer | ? |
| Customer Value | Feature | Continuous | *The calculated value of customer* |
| Churn	| Target | Binary | Binary class label (0: non-churn, 1: churn) |

### Data ETL

Extract, transform, load (ETL) is a three-phase computing process where data is:
1. extracted from an input source: dalam project ini, data diektraksi langsung dari sumbernya menggunakan library yang disediakan oleh penyedia dataset, 
2. transformed: karena dataset telah tersedia dalam format DataFrame, maka tahapan ini cukup menggabungkan variabel dataset yang terpisahkan antara feature dengan target, and 
3. loaded into an output data container: berupa variable instance dalam bahasa pemrograman Python.

In [76]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
  
iranian_churn = fetch_ucirepo(id=563) 
  
X = iranian_churn.data.features 
y = iranian_churn.data.targets 

dataset = pd.concat(
    objs=[X, y],
    axis=1
)

dataset.head(n=5)

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,Churn
0,8,0,38,0,4370,71,5,17,3,1,1,30,197.640,0
1,0,0,39,0,318,5,7,4,2,1,2,25,46.035,0
2,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520,0
3,10,0,38,0,4198,66,1,35,1,1,1,15,240.020,0
4,3,0,38,0,2393,58,2,33,1,1,1,15,145.805,0


* data succesfully extracted from the input source, transformed to the appropriate format, and loaded into the variable instance
* dan ditampilkan 5 baris data pertama dari dataset sebagai contoh hasil dari tahapan ETL
* setiap baris dalam dataframe merepresentasikan satu *instance* dari pelanggan Telecom
* sedangkan kolom adalah variabel atau *feature* yang mewakili keragaman dari tiap pelanggan
* khusus untuk kolom terakhir merupakan *target*...

---

## Initial Data Analysis

Fase [data analisis awal](https://en.wikipedia.org/wiki/Data_analysis#Initial_data_analysis) akan berfokus untuk melakukan pengecekan terhadap: 
* **Karakteristik sampel data**: deskripsi dari struktur sampel data, dengan melihat statistik dasar dari important variables
* **Kualitas data**: menggunakan beragam tipe analisis, seperti frequency counts, descriptive statistics (mean, standard deviation, dan median), dan normality (skewness, kurtosis, dan frequentist test)
* **Extreme observations**: outlying observations in the data are analyzed to see if they seem to disturb the distribution.

Untuk memenuhi asumsi yang diperlukan dari model fitting dan uji hipotesis, serta melakukan transformasi terhadap data sejauh yang diperlukan.

### Data Audit

The data is audited or inspected with the use of statistical and database methods to detect anomalies and contradictions: this eventually indicates the characteristics of the anomalies and their locations.

In [77]:
df = dataset.copy()

*Copying into new dataframe*:
1. *To avoid reflecting modifications to the original dataset, and*
2. *To avoid [chained indexing](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-view-versus-copy).*

In [78]:
def audit_data(data):
    audit_description = {
        "Row/Instance": data.shape[0],
        "Column/Variable": data.shape[1],
        "Integer Variable": len(data.select_dtypes(include="int64").columns),
        "Continuous Variable": len(data.select_dtypes(include="float64").columns),
        "Boolean Variable": len(data.select_dtypes(include="bool").columns),
        "Categorical Variable": len(data.select_dtypes(include="object").columns)
        }
    
    audit_result = {
        "Dtype": data.dtypes,
        "Duplicated": data.duplicated().sum(),
        "Missing": data.isna().sum(),
        "Unique": data.nunique(),
        "Sample": [data[column].unique() for column in data.columns]
        }
    
    return pd.DataFrame(data=audit_description, index=["Value"]).T,\
        pd.DataFrame(data=audit_result).rename_axis(index="Variable")

df_description, df_result = audit_data(data=df)

print("Data Description:")
display(df_description)
print("Data Audit:")
display(df_result)

Data Description:


,Value
Row/Instance,3150
Column/Variable,14
Integer Variable,13
Continuous Variable,1
Boolean Variable,0
Categorical Variable,0


Data Audit:


,Dtype,Duplicated,Missing,Unique,Sample
Variable,,,,,
Call Failure,int64,300,0,37,"[8, 0, 10, 3, 11, 4, 13, 7, 6, 9, 25, 2, 23, 2..."
Complains,int64,300,0,2,"[0, 1]"
Subscription Length,int64,300,0,45,"[38, 39, 37, 33, 36, 34, 35, 31, 27, 26, 25, 1..."
Charge Amount,int64,300,0,11,"[0, 1, 2, 3, 8, 4, 9, 7, 5, 10, 6]"
Seconds of Use,int64,300,0,1756,"[4370, 318, 2453, 4198, 2393, 3775, 2360, 9115..."
Frequency of use,int64,300,0,242,"[71, 5, 60, 66, 58, 82, 39, 121, 169, 83, 95, ..."
Frequency of SMS,int64,300,0,405,"[5, 7, 359, 1, 2, 32, 285, 144, 0, 8, 54, 483,..."
Distinct Called Numbers,int64,300,0,92,"[17, 4, 24, 35, 33, 28, 18, 43, 44, 25, 12, 32..."
Age Group,int64,300,0,5,"[3, 2, 1, 4, 5]"


* terdapat `3150` baris dalam dataset yang ditunjukkan dalam `Data Description`, baik yang berasal dari feature variable `X` maupun target variable `y`
* jumlah `14` kolom feature sesuai dengan yang telah dipaparkan dalam bagian `about dataset`, termasuk 1 kolom yang merupakan target variable 
* nama dari `14` kolom tersebut tertera dalam indeks `variable` pada `Data Audit`
* `int64` dan `float64` merupakan 2 tipe data di dalam dataset
    * Jumlah dari keempat tipe data tersebut ditunjukkan dalam DataFrame di atas:
        1. `int64`: `13` kolom
        2. `float64`: `1` kolom
    * beberapa tipe data sudah sesuai dengan random `Sample` yang ditunjukkan pada tiap variabel. semisal `Call Failure` yang memang berisikan bilangan bulat dan `Customer Value` berisikan bilangan desimal
    * namun pada variabel `Complains`, `Tariff Plan`, `Status`, dan `Churn`, variabel ini sebenarnya adalah representasi dari [binary attribute/data](https://en.wikipedia.org/wiki/Binary_data). pun demikian pada variabel `Charge Amount`, `Age Group`, dan `Age`, yang sebenarnya merepresentasikan [ordinal attribute/data](https://en.wikipedia.org/wiki/Ordinal_data)
        * sehingga tidak seharusnya tipe data dari variabel tersebut berupa integer
        * hal ini dikarenakan by [default](https://pandas.pydata.org/docs/user_guide/basics.html#defaults), any integer data will be casted as `int64` and float data as `float64` by Pandas
* terdeteksi `300` baris data yang terduplikasi, sebagaimana yang ditunjukkan pada nilai kolom `Duplicated` untuk seluruh variabel
    * satu alasan mengapa hal ini terjadi adalah tidak adanya unique identifier atau primary key yang membedakan tiap instance/baris data yang merepresentasikan satu pelanggan Telecom dengan pelanggan lainnya
    * pengaruh data yang terduplikasi pada bagian data analisis?
* tidak ada missing data yang terdeteksi, sebagaimana yang ditunjukkan pada kolom `Missing` yang bernilai `0` untuk seluruh variabel
* rentang nilai atau magnitude pada variabel numerikal bervariasi
    * mulai dari rentang satuan dan puluhan yang dicontohkan pada `Sample` variabel `Call Failure` (misal nilai `8` dan `10`) hingga ratusan dan ribuan pada variabel `Seconds of Use` (misal nilai `318` dan `4370`)
    * pengaruh magnitude pada data analisis?
* jumlah nilai atau kardinalitas pada variabel yang seharusnya bernilai kategorikal (variabel ordinal) masih bernilai wajar, sebagaimana yang ditunjukkan pada kolom `Unique` untuk feature `Age Group` dan `Age` yang bernilai `5` hingga feature `Charge Amount` yang bernilai `11`


| Anomaly | Insight | Action |
| --- | --- | --- |
| Data Type | **Data type issue detected** | Handled in `Data Cleaning` section |
| Duplicated Data | **Duplicated data detected** | Handled in `Data Cleaning` section |
| Missing Data | No missing data detected | No action needed |
| Variable Magnitude | **Variable magnitude issue detected** | ... |
| Cardinality | No cardinality issue detected | No action needed |

### Data Cleaning

Data cleansing or data cleaning is the process of identifying and correcting (or removing) corrupt, inaccurate, or irrelevant records from a dataset, table, or database. 
* It involves detecting incomplete, incorrect, or inaccurate parts of the data and then replacing, modifying, or deleting the affected data.
* The inconsistencies detected or removed may have been originally caused by user entry errors, by corruption in transmission or storage, or by different data dictionary definitions of similar entities in different stores.
* tahapan ini adalah penerapan dari [validity](https://en.wikipedia.org/wiki/Data_cleansing#Data_quality) dalam data science, karena: 
    1. values in a particular column must be of a particular data type, e.g., Boolean, numeric (integer or real), date.
    2. duplicated data...

#### Variable Name

sebelum memulai tahapan pembersihan data, perlu diperhatikan nama variabel dalam dataset sebagai berikut:

In [79]:
df.columns.to_list()

['Call  Failure',
 'Complains',
 'Subscription  Length',
 'Charge  Amount',
 'Seconds of Use',
 'Frequency of use',
 'Frequency of SMS',
 'Distinct Called Numbers',
 'Age Group',
 'Tariff Plan',
 'Status',
 'Age',
 'Customer Value',
 'Churn']

* nama variabel `Call  Failure`, `Subscription  Length`, dan `Charge  Amount` ternyata terpisahkan dengan dua karakter spasi
* untuk menjaga konsistensi penamaan variabel serta representasi dengan data dictionary, maka tahapan berikutnya akan:
    1. memisahkan string `str` object dalam tiap nama kolom
    2. menggabungkan string tersebut hanya dengan satu karakter spasi

In [80]:
df_columns_splitted = df.columns.str.split()
df.columns = df_columns_splitted.str.join(sep=" ")
df.columns.to_list()

['Call Failure',
 'Complains',
 'Subscription Length',
 'Charge Amount',
 'Seconds of Use',
 'Frequency of use',
 'Frequency of SMS',
 'Distinct Called Numbers',
 'Age Group',
 'Tariff Plan',
 'Status',
 'Age',
 'Customer Value',
 'Churn']

#### Data Type

* untuk `Complains` dan `Churn`, dua variabel ini dapat diubah tipe datanya menjadi `bool`, karena nilai yang terkandung dalam variabel ini sudah sesuai dengan *truth values* dalam [Boolean algebra](https://en.wikipedia.org/wiki/Boolean_algebra)
* sedangkan `Tariff Plan` dan `Status` akan diubah menjadi tipe data kategorikal `object`, untuk menjaga original values di dalam data dan juga representasi yang sama dengan data dictionary
* variabel ordinal `Charge Amount`, `Age Group`, dan `Age` juga akan diubah menjadi kategorikal

In [81]:
boolean_variables = ["Complains", "Churn"]
categorical_variables = ["Tariff Plan", "Status", "Charge Amount", "Age Group", "Age"]

for variable in boolean_variables:
    df[variable] = df[variable].astype(dtype="bool")
for variable in categorical_variables:
    df[variable] = df[variable].astype(dtype="object")

df.dtypes.to_frame(name="Dtype")

,Dtype
Call Failure,int64
Complains,bool
Subscription Length,int64
Charge Amount,object
Seconds of Use,int64
Frequency of use,int64
Frequency of SMS,int64
Distinct Called Numbers,int64
Age Group,object
Tariff Plan,object


#### Duplicated Data

In [82]:
df[df.duplicated()]

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,Churn
518,0,False,37,0,0,0,0,0,2,1,2,25,0.00,True
628,0,False,35,0,0,0,0,0,2,1,2,25,0.00,True
718,0,False,37,0,0,0,0,0,2,1,2,25,0.00,True
728,0,False,36,0,0,0,0,0,3,1,2,30,0.00,True
901,0,False,38,0,0,0,0,0,2,1,2,25,0.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,21,False,19,2,6697,147,92,44,2,2,1,25,721.98,False
3146,17,False,17,1,9237,177,80,42,5,1,1,55,261.21,False
3147,13,False,18,4,3157,51,38,21,3,1,1,30,280.32,False
3148,7,False,11,2,4695,46,222,12,3,1,1,30,1077.64,False


* tampak rincian dari 300 baris data yang terduplikasi sesuai paparan pada bagian sebelumnya
* baris data ini memiliki duplikat pada $ 3150 - 300 = 2850 $ baris data lainnya (yang akan ditunjukkan berikutnya setelah menghapus duplicated data dari dataset)
* data yang terduplikasi akan dihapus dengan menyisakan the first occurences

In [83]:
df = df.drop_duplicates(keep="first")
df

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,Churn
0,8,False,38,0,4370,71,5,17,3,1,1,30,197.640,False
1,0,False,39,0,318,5,7,4,2,1,2,25,46.035,False
2,10,False,37,0,2453,60,359,24,3,1,1,30,1536.520,False
3,10,False,38,0,4198,66,1,35,1,1,1,15,240.020,False
4,3,False,38,0,2393,58,2,33,1,1,1,15,145.805,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,17,False,17,1,9237,177,80,42,5,1,1,55,261.210,False
3097,13,False,18,4,3157,51,38,21,3,1,1,30,280.320,False
3099,8,True,11,2,1792,25,7,9,3,1,1,30,100.680,True
3123,26,False,38,9,10740,99,8,44,5,2,1,55,174.585,False


#### Summary

In [84]:
df_cleaned_description, df_cleaned_result = audit_data(data=df)

print("Data Description:")
display(df_cleaned_description)
print("Data Audit:")
display(df_cleaned_result)

Data Description:


,Value
Row/Instance,2850
Column/Variable,14
Integer Variable,6
Continuous Variable,1
Boolean Variable,2
Categorical Variable,5


Data Audit:


,Dtype,Duplicated,Missing,Unique,Sample
Variable,,,,,
Call Failure,int64,0,0,37,"[8, 0, 10, 3, 11, 4, 13, 7, 6, 9, 25, 2, 23, 2..."
Complains,bool,0,0,2,"[False, True]"
Subscription Length,int64,0,0,45,"[38, 39, 37, 33, 36, 34, 35, 31, 27, 26, 25, 1..."
Charge Amount,object,0,0,11,"[0, 1, 2, 3, 8, 4, 9, 7, 5, 10, 6]"
Seconds of Use,int64,0,0,1756,"[4370, 318, 2453, 4198, 2393, 3775, 2360, 9115..."
Frequency of use,int64,0,0,242,"[71, 5, 60, 66, 58, 82, 39, 121, 169, 83, 95, ..."
Frequency of SMS,int64,0,0,405,"[5, 7, 359, 1, 2, 32, 285, 144, 0, 8, 54, 483,..."
Distinct Called Numbers,int64,0,0,92,"[17, 4, 24, 35, 33, 28, 18, 43, 44, 25, 12, 32..."
Age Group,object,0,0,5,"[3, 2, 1, 4, 5]"


* terdapat `2850` baris dalam dataset yang telah dibersihkan
* `14` kolom/variabel dalam dataset yang telah dibersihkan terdiri dari:
    1. Integer `int64`: `6` kolom
    2. Continuous `float64`: `1` kolom
    3. Boolean `bool`: `2` kolom
    4. Categorical `object`: `5` kolom
* tidak ada lagi data yang terduplikasi, sebagaimana yang ditunjukkan pada kolom `Duplicated` dari `Data Audit` pada tiap variabel bernilai `0`